In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import sys

import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

DATASET_PATH = Path("./data/text")

VOC_SIZE = 1000

def load_data(datapath, max_size=None):
    texts_files = list(datapath.glob("*.txt"))
    texts = []  
    for files in texts_files:
        with open(files, "r") as files:
            text = files.readlines()
            texts += text
    texts = list(set(texts))
    return texts

texts = load_data(DATASET_PATH)

from tokenizers import Tokenizer
from transformers import BertTokenizer, BertForMaskedLM
from tokenizers.models import WordPiece
from tokenizers.trainers import WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.pre_tokenizer = Whitespace()
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

inputs = tokenizer(texts, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

inputs['labels'] = inputs.input_ids.detach().clone()

rand = torch.rand(inputs.input_ids.shape)

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'c

In [ ]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

[11, 24, 40, 41, 48]

In [18]:
selection = []

for i in range(mask_arr.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
        )

selection[:5]

for i in range(mask_arr.shape[0]):
    inputs.input_ids[i, selection[i]] = 103 # application du token [MASK]

class DATASET_EXEMPLE(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
        
    def __getitem__(self, idx):
        return {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
    
    def __len__(self):
        return len(self.encodings.input_ids)

dataset = DATASET_EXEMPLE(inputs)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [24]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model.to(device)

In [28]:
model.train()

from transformers import AdamW

optim = AdamW(model.parameters(), lr=1e-4)

from tqdm import tqdm

epochs = 2

for epoch in range(epochs):
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        
        loss = outputs.loss
        loss.backward()
        optim.step()
        
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

c:\Users\rapha\miniconda3\envs\DL_IODAA\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1094 [00:00<?, ?it/s]C:\Users\rapha\AppData\Local\Temp\ipykernel_11864\2832896330.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
  0%|          | 0/1094 [01:18<?, ?it/s]


KeyboardInterrupt: 